# Störmer's Method

As a demonstration how *findiff* can be used to symbolically create iteration
algorithms, we will apply it to the simple ordinary differential
equation of a harmonic oscillator:

$$
u'' + \omega^2 u = 0
$$

with the initial conditions $u(0) = a$ and $u'(0) = 0$.

If you want to follow along, install *findiff* and start up iPython or a Jupyter notebook.
We will need the `Stencil` class and some features for symbolic operations contained in the
`findiff.symbolics` subpackage.

In [1]:
from findiff import Stencil
from findiff.symbolics import Equation, Symbol

All symbolic features of *findiff* are based on *sympy*. In fact,
`Symbol` is identical with the *sympy* version of `Symbol`. It is just included so the users
don't have to import if from *sympy* manually. The `Equation` class is a subclass of *sympy*'s
`Eq` class, but it has several additional features convenient for the present use case
compared to the parent class.

Next, we will discretize the second derivative $u''$ using a central scheme.

In [2]:
stencil = Stencil(
    offsets=[-1, 0, 1],
    partials={(2,): 1},
    spacings=[r'\Delta t'],
    symbolic=True
)

Now we can turn the stencil into a symbolic expression. By default, the name of the grid function is $u$,
but this can be adjusted.

In [3]:
expr, symbols = stencil.as_expression(index_symbols='n')

This returns the expression and also a dictionary with the symbols, so that we can use them
in our code.

In [4]:
expr

u[n + 1]/\Delta t**2 + u[n - 1]/\Delta t**2 - 2*u[n]/\Delta t**2

In [5]:
symbols

{'indices': [n], 'function': u, 'spacings': [\Delta t]}

In [6]:
n, = symbols['indices']
u = symbols['function']
dx, = symbols['spacings']

Now we define our discretized equation:

In [7]:
w = Symbol(r'\omega')
eq = Equation(expr + w**2 * u[n], 0)
eq

Eq(\omega**2*u[n] + u[n + 1]/\Delta t**2 + u[n - 1]/\Delta t**2 - 2*u[n]/\Delta t**2, 0)

and solve it for $u_{n+1}$:

In [8]:
eq.solve(u[n+1])

Eq(u[n + 1], -\Delta t**2*\omega**2*u[n] - u[n - 1] + 2*u[n])

In [9]:
import findiff.symbolics as sym
sym.collect(_, u[n])

Eq(u[n + 1], (-\Delta t**2*\omega**2 + 2)*u[n] - u[n - 1])

Now we have $u_{n+1}$ in terms of the previous grid points. So we have an explicit iteration scheme.
This scheme is sometimes called Störmer's method.

In [10]:
stoermer = _

But there is still one issue to solve. Look at what happens for $n=0$:

In [11]:
stoermer.subs({n: 0})

Eq(u[1], (-\Delta t**2*\omega**2 + 2)*u[0] - u[-1])

To determine $u_1$ we would need $u_{-1}$, which does not even exist. But fortunately,
we can get rid of it using the initial condition $u'(0)=0$. Discretize that condition:

In [12]:
stencil = Stencil(offsets=[-1, 1], partials={(1,):1}, spacings=[dx], symbolic=True)
expr, symbols = stencil.as_expression(index_symbols=['n'])
eq = Equation(expr.subs(n, 0), 0)
eq

Eq(-u[-1]/(2*\Delta t) + u[1]/(2*\Delta t), 0)

Solve that for $u_{-1}$

In [13]:
eq = eq.solve(u[-1])
eq

Eq(u[-1], u[1])

And finally substitute that in our expression for the first step:

In [14]:
stoermer.subs({n: 0}).subs(eq.as_subs())

Eq(u[1], (-\Delta t**2*\omega**2 + 2)*u[0] - u[1])

In [15]:
_.solve(u[1])

Eq(u[1], (-\Delta t**2*\omega**2 + 2)*u[0]/2)

Now we have all we need:

- Having $u_0$ (the initial condition!), obtain the first step $u_1$
- For $n>1$, determine $u_{n+1}$ from the iteration scheme

In [16]:
stoermer

Eq(u[n + 1], (-\Delta t**2*\omega**2 + 2)*u[n] - u[n - 1])